## 4 对象连接

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('test.csv')
data.head()

,date,temperature,load
0,12/08/19,25.0,1031
1,12/09/19,22.0,1081
2,12/10/19,27.0,1063
3,12/11/19,28.0,1090
4,12/12/19,21.0,1005


In [4]:
tmp = data.head()
tmp.index=list('abcde')       # 索引修改
tmp.index.name = 'letter'     # 索引名修改
tmp

,date,temperature,load
letter,,,
a,12/08/19,25.0,1031
b,12/09/19,22.0,1081
c,12/10/19,27.0,1063
d,12/11/19,28.0,1090
e,12/12/19,21.0,1005


In [5]:
tmp.loc['f'] = ['12/13/19', 37, 1212]    # 插入一行记录
tmp

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,date,temperature,load
letter,,,
a,12/08/19,25.0,1031
b,12/09/19,22.0,1081
c,12/10/19,27.0,1063
d,12/11/19,28.0,1090
e,12/12/19,21.0,1005
f,12/13/19,37.0,1212


In [16]:
df = data.rename(columns={'date':'date_a', 'temperature': 'temp',\
                         'load': 'load_a'})
df.head()

,date_a,temp,load_a
0,12/08/19,25.0,1031
1,12/09/19,22.0,1081
2,12/10/19,27.0,1063
3,12/11/19,28.0,1090
4,12/12/19,21.0,1005


## concat, join和merge的区别

In [9]:
attention = '''
1 concate函数：
  a 可以水平和垂直的连接两个或多个pandas对象
  b 只用索引对齐，索引出现重复时会报错
  c 默认为外连接，也可设置为内连接
  
2 join函数：
  a 只能水平连接两个或多个pandas对象
  b 对齐是靠被调用的dataframe的列索引或行索引和另一个dataframe的行索引（不能靠列索引）
  c 通过笛卡尔积处理重复的索引值
  d 默认为左连接，也可设置为右连接、外连接和内连接
  
3 merge函数（与join函数功能类似，只是仅针对dataframe）：
  a 只能水平连接两个dataframe对象
  b 对齐是靠被调用的dataframe的列或行索引和另一个dataframe的列或行索引
  c 通过笛卡尔积处理重复的索引值
  d 默认为内连接，也可设置为外连接，左连接和右连接
'''

In [11]:
# merge函数
df1 = pd.DataFrame({'key':['X','Y','Z','X','Z'], 'vals1': np.arange(5)})
df1

,key,vals1
0,X,0
1,Y,1
2,Z,2
3,X,3
4,Z,4


In [17]:
df2 = pd.DataFrame({'key':['A','B','X'], 'vals1':[1,2,3], 'vals2':[9,9,9]})
df2

,key,vals1,vals2
0,A,1,9
1,B,2,9
2,X,3,9


In [18]:
# 使用默认参数的merge函数时，merge会自动寻找两个dataframe中的相同列，
# 并以inner join的形式合并
pd.merge(df1, df2)   # 默认为‘inner’连接

,key,vals1,vals2
0,X,3,9


In [19]:
pd.merge(df1, df2, on=['key', 'vals1'], how='inner')

,key,vals1,vals2
0,X,3,9


In [20]:
pd.merge(df1, df2, on='key', how='inner')   # 内连接是取on指定列的公共行

,key,vals1_x,vals1_y,vals2
0,X,0,3,9
1,X,3,3,9


In [21]:
# 左连接
pd.merge(df1, df2, on='key', how='left')    # 像df1对齐

,key,vals1_x,vals1_y,vals2
0,X,0,3.0,9.0
1,Y,1,NaN,NaN
2,Z,2,NaN,NaN
3,X,3,3.0,9.0
4,Z,4,NaN,NaN


In [22]:
# 右连接
pd.merge(df1, df2, on='key', how='right')   # 像df2对齐

,key,vals1_x,vals1_y,vals2
0,X,0.0,3,9
1,X,3.0,3,9
2,A,NaN,1,9
3,B,NaN,2,9


In [23]:
# 外/全连接
pd.merge(df1, df2, on='key', how='outer')     # 将key中在两个df中的所有值全部显示出来

,key,vals1_x,vals1_y,vals2
0,X,0.0,3.0,9.0
1,X,3.0,3.0,9.0
2,Y,1.0,NaN,NaN
3,Z,2.0,NaN,NaN
4,Z,4.0,NaN,NaN
5,A,NaN,1.0,9.0
6,B,NaN,2.0,9.0


In [25]:
# 以索引作为拼接参考
pd.merge(df1, df2, left_on='vals1', right_index=True)

,vals1,key_x,vals1_x,key_y,vals1_y,vals2
0,0,X,0,A,1,9
1,1,Y,1,B,2,9
2,2,Z,2,X,3,9


In [39]:
# concat连接
ser1 = pd.Series([0,2,1], index=['T', 'U', 'V'])
ser2 = pd.Series([3,4,5], index=['A', 'B', 'V'])
ser1, ser2

(T    0
 U    2
 V    1
 dtype: int64, A    3
 B    4
 V    5
 dtype: int64)

In [40]:
pd.concat([ser1, ser2], axis=0, sort=True)  # 默认axis=0, sort=True

T    0
U    2
V    1
A    3
B    4
V    5
dtype: int64

In [41]:
pd.concat([ser1, ser2], axis=1, join_axes=[['T', 'U', 'V']])

,0,1
T,0,NaN
U,2,NaN
V,1,5.0
